**Задание 1**

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import datetime
import pandas as pd

In [2]:
KEYWORDS = ['python', 'парсинг']
url = 'https://habr.com/ru/all/'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')
titles = list(map(lambda x: x.find('h2', class_='post__title'), posts))
links = list(map(lambda x: x.find('a').get('href'), titles))


def get_all_links(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    titles = list(map(lambda x: x.find('h2', class_='post__title'), posts))
    links = list(map(lambda x: x.find('a').get('href'), titles)) 
    return links

all_links = get_all_links(url)



def get_news(links, keywords):
    all_art = pd.DataFrame()
    for link in links:
        req = requests.get(link).text
        soup = BeautifulSoup(req, 'html.parser')
        time.sleep(0.3)
        art = soup.find('div', class_='post__body').text.lower()
        for word in keywords:
            if word in art:         
                date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
                title = soup.find('span', class_='post__title-text').text
                res = {'date': date, 'title': title, 'link': link, 'text': art}
                all_art = pd.concat([all_art, pd.DataFrame([res])])
    return all_art

all_art = get_news(all_links, KEYWORDS)
all_art


,date,title,link,text
0,2021-01-19,Сквозная Аналитика на Azure SQL + dbt + Github...,https://habr.com/ru/post/538106/,"\n\nпривет, хабр! меня зовут артемий козырь.\n..."
0,2021-01-19,Как новые квантовые алгоритмы решили нелинейны...,https://habr.com/ru/company/skillfactory/blog/...,\nиногда компьютеру легко спрогнозировать буду...
0,2021-01-19,Настройка аутентификации JWT в новом проекте D...,https://habr.com/ru/post/538040/,\nданная статья является сборкой-компиляцией н...


**Задание 2**

In [3]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
email = ['xxx@x.ru', 'yyy@y.com']

def get_req(url, emails):
    list_req = []
    headers = {
        'Vaar-Version': '0',
        'Vaar-Header-App-Product': 'hackcheck-web-avast'  
    }
    for e in emails:
        data = {
            'emailAddresses': [e]
        }
        req = requests.post(url, json=data, headers=headers)
        list_req.append(req.json())
        time.sleep(3)
    return list_req

list_req = get_req(url, email)
list_req 

[{'breaches': {'3176': {'breachId': 3176,
    'site': 'parapa.mail.ru',
    'recordsCount': 5029003,
    'description': "In July and August 2016, two criminals executed attacks against three separate forums hosted by Mail.ru including the Russian forum Parapa. Shortly after the breach occurred, the contents of Parapa's database were leaked publicly. The database contains usernames, email addresses, and hashed passwords for around 5 million users.",
    'publishDate': '2017-02-14T00:00:00Z',
    'statistics': {'usernames': 5029000,
     'passwords': 5029003,
     'emails': 4941344}},
   '12': {'breachId': 12,
    'site': 'vk.com',
    'recordsCount': 110121799,
    'description': "Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on 

In [7]:
def get_leakes(dict_breaches, emails):
    leakes_info = pd.DataFrame()
    for i in range(len(dict_breaches)):
        leak = dict_breaches[i]['breaches']
        email = emails[i]
        for key, value in leak.items():
            date = pd.to_datetime(value['publishDate']).date()
            source = value['site']
            descr = value['description']
            row = {'email': email, 'breaches_date': date, 'source_leak': source, 'description': descr}
            leakes_info = pd.concat([leakes_info, pd.DataFrame([row])])
    return leakes_info

leakes_info = get_leakes(list_req, email)
leakes_info

,email,breaches_date,source_leak,description
0,xxx@x.ru,2017-02-14,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,xxx@x.ru,2016-10-29,vk.com,Popular Russian social networking platform VKo...
0,xxx@x.ru,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,xxx@x.ru,2017-02-14,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,xxx@x.ru,2017-01-31,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
0,xxx@x.ru,2016-10-23,imesh.com,"In June 2016, a cache of over 51 million user ..."
0,yyy@y.com,2020-01-03,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
0,yyy@y.com,2020-05-28,wishbone.io,"In January 2020, the online poll website Wishb..."
0,yyy@y.com,2017-11-04,myheritage.com,"In October 2017, a customer database belonging..."
0,yyy@y.com,2019-06-13,canva.com,"In May 2019, graphic-design site Canva's datab..."
